In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("EmotionCSV.csv")
#df2 = pd.read_csv("./Training_Dataframes/esken30ktweetswithemotions.csv")

In [ ]:
#print(type(df))
#print(df.head())
df.drop(["Unnamed: 0"],axis = 1, inplace = True)
#print("----")

In [ ]:
#print(df.head())
print(df.keys())
keyString = df.keys()
keyString += "Input"
    
df.columns = keyString
print(df.keys())

In [ ]:
df.keys()

In [ ]:
listOfTweets = df["editedInput"].tolist()
docs = np.array(listOfTweets)

"""
#Getting amount of individual words
longListofWords = " ".join(docs).split()
#print(longListofWords[0])
unique = set(longListofWords)
countOfUniqueWords = len(unique)
print(countOfUniqueWords)"""

In [ ]:
# https://medium.com/swlh/tweet-sentiment-analysis-using-python-for-complete-beginners-4aeb4456040
from sklearn.feature_extraction.text import CountVectorizer
#dir(CountVectorizer)

In [ ]:
cv=CountVectorizer(binary = True)
#help(cv.fit_transform)
oneHotCv = cv.fit_transform(docs)
#print(type(oneHotCv))
#print(dir(oneHotCv))
#print(cv.get_feature_names())
oneHotArray = oneHotCv.toarray()
oneHotArray.shape

In [ ]:
"""
numberZero = 0
numberOne = 1
print("Length of Tweet {} according to textforttb: ".format(numberZero)+str(len(df["edited"][numberZero].split()))) 
print("Length of Tweet {} according to oneHotArray: ".format(numberZero)+str(sum(oneHotArray[numberZero])))
print(max(oneHotArray[0]))
print(df["edited"][numberZero].split())
print(oneHotArray[numberZero])
# All Unique Words are represented in the oneHotArray. Only "Und" ist removed, which results in a length -1.
"""

In [ ]:
oneHotDf = pd.DataFrame(oneHotArray)
oneHotDf.columns = cv.get_feature_names_out()
print(oneHotDf.head())
print(oneHotDf.info())
#print(oneHotDf["text"])

In [ ]:
longDF = pd.concat([df,oneHotDf], axis = 1)
#print(type(longDF))


In [ ]:
#print(df.info())
print(longDF.info())
print(str(longDF.count()))
#dir(longDF)

In [ ]:
print(longDF.iloc[0]["editedInput"])
print(longDF.iloc[0][longDF.iloc[0]["editedInput"].split()])

In [ ]:
print(longDF.keys()[:27])
print(longDF.keys()[:24])
#Shows that keys[0:25] are tweet data, thats not part of the NLP

In [ ]:
#This is already part of old code
import re

def clean(text):
    text = re.sub(r'@[A-Za-z0-9]+\s?', '', text) #Removed Mentions
    text = re.sub(r'#', '', text) #Removed #
    text = re.sub(r'(.)1+', r'1', text) #cleaned single letters
    text = re.sub('((www.[^s]+)|(https?://[^s]+))','',text) #Removes links
    text = re.sub('@','',text) #Remove @
    text = re.sub('-','',text) #Remove -
    text = re.sub('ä','ae',text) #Remove ä
    text = re.sub('Ä','Ae',text) #Remove Ä
    text = re.sub('ö','oe',text) #Remove Ä
    text = re.sub('Ö','Oe',text) #Remove Ä
    text = re.sub('ü','ue',text) #Remove Ä
    text = re.sub('Ü','Ue',text) #Remove Ä
    return text

import nltk
from nltk.stem import PorterStemmer
porter = PorterStemmer()
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data

In [ ]:
def sentence_toVec(sentence,goalDF):
    #Preparing Input text
    cleanSentence = sentence.lower()
    cleanSentence = clean(cleanSentence)
    cleansentene = stemming_on_text(cleanSentence)
    
    #Preparing EmptyDF
    emptyList = [0]*len(goalDF.keys())
    #print(emptyList)
    #print("Len EmptyList: " + str(len(emptyList)))
    #emptyDF = goalDF.iloc[0:0].copy()
    #emptyDF = emptyDF.append(emptyList)
    #emptyDF = emptyDF.append(pd.DataFrame(emptyList, columns = goalDF.keys()), ignore_index = True)
    emptyDF = pd.DataFrame(columns = goalDF.keys())
    #emptyDF.append(pd.Series(), ignore_index = True)
    emptyDF.loc[len(emptyDF)] = emptyList
    #print(emptyDF)
    #emptyDF = pd.DataFrame(emptyList, columns = goalDF.keys())
    print("LenEmpty DF: " + str(len(emptyDF)) + " Len Keys Empty DF: " + str(len(emptyDF.keys())))
    
    emptyDF["textInput"][0] = sentence
    emptyDF["editedInput"][0] = cleanSentence
    #keys = list(emptyDF.keys()[0:10])
    #print("---")
    print("New LenEmpty DF: " + str(len(emptyDF)) + " Len Keys Empty DF: " + str(len(emptyDF.keys())))
    #print(emptyDF["textInput"].tolist())
    listOfWords = cleanSentence.split()
    for word in listOfWords:
        if word in emptyDF.columns:
            emptyDF[word][0] = 1
    return emptyDF
    
    
    

In [ ]:
longDF.keys()[100:120]

In [ ]:
dfOfSentence = sentence_toVec("Wir sind heute leider alle sehr sauer. Daher lasst uns alle feiern! Zusammen sind wir stark aber auch bischen wütend", longDF)
#print(dfOfSentence.loc[:,["textInput","editedInput","WutInput"]])
listOfIndex = dfOfSentence.columns[(dfOfSentence == 1).all()].tolist()
#print("ListofIndex" + str(listOfIndex))

listOfIndex.extend(["textInput","editedInput","WutInput"])
#print("This is a new List: " + str(listOfIndex))
print(dfOfSentence.loc[:,listOfIndex])

In [ ]:
#emotion_Model = DecisionTreeRegressor(random_state = 1)
#help(emotion_Model.predict)

In [ ]:
tweet_features = longDF.keys()[24:]
"""print(df.keys())
print(str(longDF["text"].head())+"\n")
print(longDF["time"].head())
print(longDF["date"].head())
print(longDF.keys()[5:].tolist().index("time")+1)
#tweet_features = longDF.keys()[10620:10630]
print(longDF[longDF.columns[10620:10630]])"""
X_trainingData = longDF[tweet_features]


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

#train_X, val_X, train_y, val_y = train_test_split(X_trainingData, y_emotionWut, random_state = 0)
emotion_Model = DecisionTreeRegressor(random_state = 1)



In [ ]:
from sklearn.metrics import mean_absolute_error
#val_predictions = emotion_Model.predict(val_X)
#print(emotion_Model.predict(val_X.head()))
#print(mean_absolute_error(val_y,val_predictions))

In [ ]:
#help(train_test_split)
#Parameters train_test_split(test_size,train_size,random_state,shuffle)
#help(DecisionTreeRegressor)
#dir(DecisionTreeRegressor)
#Parameters DecisionTreeRegressor(criterion,splitter,max_depth,min_sample_split,min_sample_leaf,min_weight_fraction_leaf,max_features,random_state,max_leaf_nodes,min_impurity_decrease)
# criterion : {"squared_error", "friedman_mse", "absolute_error","poisson"}, default="squared_error"
#help(emotion_Model.score)
#dir(longDF)

In [ ]:
keys = list(longDF.keys()[12:20])
#print(list((keys)))
for emotion in longDF[keys]:
    print(emotion[:-5] + ": ")
    #print(longDF[emotion].head())
    y_emotion = longDF[emotion]
    train_X, val_X, train_y, val_y = train_test_split(X_trainingData, y_emotion,test_size = 0.33, random_state = 1,shuffle=True)
    emotion_Model = DecisionTreeRegressor(random_state = 1)
    emotion_Model.fit(X_trainingData,y_emotion)
    val_predictions = emotion_Model.predict(val_X)
    #print(emotion_Model.predict(val_X.head()))
    print("Mean Absolute error for {}: ".format(emotion[:-5]) + str(round(mean_absolute_error(val_y,val_predictions)*100,2))+"%\n")
    
    
    
    print("type(train_X): " + str(type(train_X)))
    print("type(dfSentence): " + str(type(dfOfSentence)))
    
    print("Predicition for 'Ich bin sehr Sauer' for train_X.head(): ".format(emotion[:-5])+ str(emotion_Model.predict(train_X.head())))
    print("Predicition for 'Ich bin sehr Sauer' for emotion {}: ".format(emotion[:-5])+ str(emotion_Model.predict(dfOfSentence[tweet_features])))
    #y_emotionGoal = longDF[emotion]
    break